<a href="https://colab.research.google.com/github/grmurad/GM_SDS-CP022-ai-travel-companion/blob/main/notebooks/AssistTravel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate bitsandbytes
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install geonamescache
!pip install amadeus
!pip install aiohttp
!pip install flask flask-ngrok
!pip install pyngrok
!pip install fuzzywuzzy python-Levenshtein

In [ ]:
from flask import Flask, request, render_template, redirect, url_for, Response
from flask_ngrok import run_with_ngrok
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import Accelerator
import torch
from amadeus import Client, ResponseError
from huggingface_hub import login
from google.colab import userdata
import re
import logging
import os
import asyncio
import aiohttp
from datetime import datetime, timedelta
import base64
from pyngrok import ngrok
from fuzzywuzzy import process
import json
import csv
import pandas as pd
import random

# Carregar a base de dados de cidades
with open("cidades_filtradas.json", "r", encoding="utf-8") as arquivo:
    BASE_CIDADES = json.load(arquivo)

def carregar_base_iata(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo, on_bad_lines='skip')
    required_columns = ["name", "iso_country", "municipality", "iata_code"]
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Coluna '{col}' não encontrada no arquivo CSV.")
    df = df.dropna(subset=['iata_code'])
    df = df[df['iata_code'] != '']
    df['municipality'] = df['municipality'].fillna('')
    base_iata = df[["name", "iso_country", "municipality", "iata_code"]].rename(columns={
        "name": "nome_aeroporto",
        "iso_country": "pais",
        "municipality": "cidade",
        "iata_code": "iata"
    }).to_dict(orient='records')
    return base_iata

BASE_IATA = carregar_base_iata("IATA_Cities.csv")

import random

# Lista de lugares famosos para turismo
LUGARES_FAMOSOS = [
    "Paris", "New York", "Tokyo", "Rome", "London",
    "Barcelona", "Dubai", "Sydney", "Rio de Janeiro", "Machu Picchu"
]

def buscar_imagem_aleatoria():
    lugar = random.choice(LUGARES_FAMOSOS)
    imagens = buscar_imagens_pixabay(lugar)
    return imagens[0] if imagens else None

    # Para requisições GET, buscar uma imagem aleatória
    imagem_fundo = buscar_imagem_aleatoria()
    return render_template("index.html", imagem_fundo=imagem_fundo)

def buscar_iata_por_cidade(nome_cidade, codigo_pais=None):
    nome_cidade = nome_cidade.strip().lower()
    resultados = []
    for cidade in BASE_IATA:
        cidade_nome = str(cidade.get("cidade", "")).lower()
        if nome_cidade in cidade_nome:
            if codigo_pais and cidade.get("pais", "").lower() != codigo_pais.lower():
                continue
            resultados.append({
                "nome": cidade["cidade"],
                "iata": cidade["iata"]
            })
    return resultados

import requests

PIXABAY_API_KEY = userdata.get('Pixabay')


def buscar_imagens_pixabay(query):
    url = f"https://pixabay.com/api/?key={PIXABAY_API_KEY}&q={query}&image_type=photo&per_page=5"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return [imagem["largeImageURL"] for imagem in data.get("hits", [])]
    return []

ngrok.set_auth_token(userdata.get('NGROK'))
app = Flask(__name__)

# Middleware to add custom User-Agent header
@app.after_request
def add_custom_user_agent(response):
    response.headers['User-Agent'] = 'Custom-Agent'
    return response

public_url = ngrok.connect(5000).public_url
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
login(token=userdata.get('HUGGINGFACE_TOKEN'))
AMADEUS_CLIENT_ID = userdata.get('AMADEUS_API_KEY')
AMADEUS_CLIENT_SECRET = userdata.get('AMADEUS_API_SECRET')

if not AMADEUS_CLIENT_ID or not AMADEUS_CLIENT_SECRET:
    raise ValueError("Amadeus credentials not found in Colab Secrets. Please add them.")

amadeus = Client(
    client_id=AMADEUS_CLIENT_ID,
    client_secret=AMADEUS_CLIENT_SECRET)

# Restante do código Flask (modelo, funções, rotas, etc.) permanece o mesmo...

async def get_session():
    return aiohttp.ClientSession()

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
accelerator = Accelerator()
model, tokenizer = accelerator.prepare(model, tokenizer)

def validate_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def extrair_cidades_e_iata(texto):
    padrao = r"([A-Za-zÀ-ÿ\s]+)\s*\(([A-Z]{3})\)"
    matches = re.findall(padrao, texto)
    cidades = [{"nome": cidade.title().strip(), "iata": iata.strip()} for cidade, iata in matches]
    return cidades

def buscar_cidades_similares_local(nome_cidade, limite=5):
    nomes_cidades = [cidade["nome"] for cidade in BASE_CIDADES]
    resultados = process.extract(nome_cidade, nomes_cidades, limit=limite)
    cidades_similares = []
    for nome, score in resultados:
        if score >= 70:
            cidade = next((c for c in BASE_CIDADES if c["nome"] == nome), None)
            if cidade:
                cidades_similares.append(cidade)
    return cidades_similares


async def buscar_detalhes_hotel(session, hotel_id, access_token):
    try:
        async with session.get(
            f"https://test.api.amadeus.com/v1/reference-data/locations/hotels/{hotel_id}",
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                hotel_details = await response.json()
                return hotel_details.get("data", {})
            else:
                logger.error(f"Failed to fetch hotel details: {response.status}")
                return None
    except Exception as e:
        logger.error(f"Erro na API Amadeus: {e}")
        return None

def get_default_departure_date():
    today = datetime.today()
    default_date = today + timedelta(days=10)
    return default_date.strftime("%Y-%m-%d")

def gerar_sugestoes_viagem(prompt_usuario, num_sugestoes=3):
    prompt_usuario += (
        f" Liste {num_sugestoes} de destinos turísticos, incluindo:"
        f" - Informe cidades com serviço regular de voos que sejam proximas às atracoes destacadas: Nome da Cidade e seu codigo IATA em parentesis: (exemplo: 'San Juan (SJU)')."
        f" - Principais atrações e atividades. use no maximo 1000 palavras de forma que resuma bem o pedido do usuario"
    )
    inputs = tokenizer(prompt_usuario, return_tensors="pt").to("cuda")
    inputs["attention_mask"] = torch.ones_like(inputs["input_ids"]).to("cuda")
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=1000,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True).strip()
    print(response)
    response = response.replace(prompt_usuario, "").strip()
    return response, extrair_cidades_e_iata(response)

async def buscar_hoteis(session, cidade_iata, access_token):
    try:
        async with session.get(
            "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city",
            params={"cityCode": cidade_iata},
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                hotel_data = (await response.json()).get("data", [])
                if not hotel_data:
                    return "No hotels found for this city."

                hotel_list = []
                for hotel in hotel_data[:5]:  # Limita a 5 hotéis
                    hotel_id = hotel.get("hotelId")
                    hotel_name = hotel.get("name", "N/A")

                    # Buscar detalhes do hotel
                    hotel_details = await buscar_detalhes_hotel(session, hotel_id, access_token)
                    if hotel_details:
                        endereco = hotel_details.get("address", {}).get("lines", ["N/A"])[0]
                        telefone = hotel_details.get("contact", {}).get("phone", "N/A")
                        preco = hotel_details.get("price", {}).get("amount", "N/A")
                    else:
                        endereco = "N/A"
                        telefone = "N/A"
                        preco = "N/A"

                    hotel_info = {
                        "nome": hotel_name,
                        "id": hotel_id,
                        "endereco": endereco,
                        "telefone": telefone,
                        "preco": preco
                    }
                    hotel_list.append(hotel_info)

                return hotel_list
            else:
                logger.error(f"Failed to fetch hotels: {response.status}")
                return None
    except Exception as e:
        logger.error(f"Erro na API Amadeus: {e}")
        return None

async def buscar_voos(session, origem_iata, destino, access_token, data_partida="2025-03-10", adultos=1):
    try:
        params = {
            "originLocationCode": origem_iata,
            "destinationLocationCode": destino,
            "departureDate": data_partida,
            "adults": adultos,
            "max": 5  # Limit to 5 results
        }
        async with session.get(
            "https://test.api.amadeus.com/v2/shopping/flight-offers",
            params=params,
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                flight_data = (await response.json()).get("data", [])
                if not flight_data:
                    logger.warning(f"No flight data found for {origem_iata} -> {destino}")
                    return None  # Retorna None se não houver voos

                # Fetch airline names
                airline_codes = set()
                for flight in flight_data:
                    for segment in flight["itineraries"][0]["segments"]:
                        airline_codes.add(segment["carrierCode"])

                # Map carrier codes to airline names
                airline_names = {}
                for code in airline_codes:
                    async with session.get(
                        "https://test.api.amadeus.com/v1/reference-data/airlines",
                        params={"airlineCodes": code},
                        headers={"Authorization": f"Bearer {access_token}"}
                    ) as airline_response:
                        if airline_response.status == 200:
                            airline_info = (await airline_response.json()).get("data", [])
                            if airline_info:
                                airline_names[code] = airline_info[0].get("businessName", code)

                # Format the flight list with more details
                flights = []
                for flight in flight_data:
                    segments = flight["itineraries"][0]["segments"]
                    flight_info = {
                        "airline": airline_names.get(segments[0].get("carrierCode", "N/A"), "N/A"),
                        "flightNumber": segments[0].get("number", "N/A"),
                        "origin": segments[0].get("departure", {}).get("iataCode", "N/A"),
                        "destination": segments[-1].get("arrival", {}).get("iataCode", "N/A"),
                        "departureTime": segments[0].get("departure", {}).get("at", "N/A"),
                        "arrivalTime": segments[-1].get("arrival", {}).get("at", "N/A"),
                        "price": flight["price"].get("total", "N/A"),
                        "currency": flight["price"].get("currency", "N/A")
                    }
                    flights.append(flight_info)
                return flights
            else:
                error_response = await response.text()
                logger.error(f"Failed to fetch flights: {response.status}, Response: {error_response}")
                return None  # Retorna None em caso de erro
    except Exception as e:
        logger.error(f"Erro na API Amadeus: {e}")
        return None  # Retorna None em caso de exceção

async def get_amadeus_token(session):
    try:
        credentials = base64.b64encode(
            f"{AMADEUS_CLIENT_ID}:{AMADEUS_CLIENT_SECRET}".encode("utf-8")
        ).decode("utf-8")
        async with session.post(
            "https://test.api.amadeus.com/v1/security/oauth2/token",
            headers={
                "Content-Type": "application/x-www-form-urlencoded",
                "Authorization": f"Basic {credentials}"
            },
            data={"grant_type": "client_credentials"}
        ) as response:
            if response.status == 200:
                token_data = await response.json()
                return token_data.get("access_token")
            else:
                error_response = await response.text()
                logger.error(f"Failed to fetch access token: {response.status}, Response: {error_response}")
                return None
    except Exception as e:
        logger.error(f"Error fetching access token: {e}")
        return None

async def fetch_hotels_and_flights(origem_iata, cidades, data_partida):
    session = await get_session()
    access_token = await get_amadeus_token(session)
    results = []
    for cidade in cidades:
        cidade_iata = cidade["iata"]
        hotels = await buscar_hoteis(session, cidade_iata, access_token)
        flights = await buscar_voos(session, origem_iata, cidade_iata, access_token, data_partida)
        results.append({
            "city": cidade["nome"],
            "iata": cidade_iata,
            "hotels": hotels if hotels else "Nenhum hotel encontrado.",
            "flights": flights if flights else "Nenhum voo encontrado."
        })
    await session.close()
    print("Flights Data:", results)  # Debug: Check the flights data
    return results

@app.route("/selecionar_cidade", methods=["POST"])
def selecionar_cidade():
    origem = request.form.get("origin")
    prompt_usuario = request.form.get("preferences")
    data_partida = request.form.get("date")
    selected_city_iata = request.form.get("selected_city")

    logger.info(f"Selected IATA code: {selected_city_iata}")
    logger.info(f"Origem: {origem}")
    logger.info(f"Prompt do usuário: {prompt_usuario}")
    logger.info(f"Data de partida: {data_partida}")

    if selected_city_iata:
        origem_iata = selected_city_iata
    else:
        resultados = buscar_iata_por_cidade(origem)
        if not resultados:
            return render_template("index.html", error=f"Não foi possível encontrar o código IATA para: {origem}", preferences=prompt_usuario, origin=origem, date=data_partida)
        if len(resultados) > 1:
            return render_template("selecionar_cidade.html", cidades=resultados, origem=origem, prompt_usuario=prompt_usuario, date=data_partida)
        origem_iata = resultados[0]["iata"]

    if not data_partida:
        data_partida = get_default_departure_date()
        logger.info(f"Nenhuma data de partida fornecida. Usando data padrão: {data_partida}")
    elif not validate_date(data_partida):
        return render_template("index.html", error="Formato de data inválido. Use AAAA-MM-DD.", preferences=prompt_usuario, origin=origem, date=data_partida)

    sugestoes, cidades = gerar_sugestoes_viagem(prompt_usuario)
    logger.info(f"Sugestões geradas: {sugestoes}")
    logger.info(f"Cidades encontradas: {cidades}")

    results = asyncio.run(fetch_hotels_and_flights(origem_iata, cidades, data_partida))
    logger.info(f"Resultados da busca: {results}")

    # Buscar imagens para cada cidade
    imagens = {}
    for cidade in cidades:
        query = cidade["nome"]
        imagens[query] = buscar_imagens_pixabay(query)
    print("Imagens encontradas:", imagens)  # Depuração
    return render_template("results.html", suggestions=sugestoes, results=results, imagens=imagens)


# Lista de lugares famosos para turismo
LUGARES_FAMOSOS = [
    "Paris", "New York", "Tokyo", "Rome", "London",
    "Barcelona", "Dubai", "Sydney", "Rio de Janeiro", "Machu Picchu", "Madrid", "Viena", "Africa", "Lisboa"
]

def buscar_imagem_aleatoria():
    lugar = random.choice(LUGARES_FAMOSOS)
    imagens = buscar_imagens_pixabay(lugar)
    return imagens[0] if imagens else None

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        origem = request.form.get("origin")
        prompt_usuario = request.form.get("preferences")
        data_partida = request.form.get("date")

        if not origem:
            return render_template("index.html", error="Por favor, forneça a cidade de origem.")

        resultados = buscar_iata_por_cidade(origem)
        if not resultados:
            return render_template("index.html", error=f"Nenhuma cidade encontrada para: {origem}", preferences=prompt_usuario, origin=origem, date=data_partida)

        if len(resultados) > 1:
            return render_template("selecionar_cidade.html", cidades=resultados, origem=origem, prompt_usuario=prompt_usuario, data_partida=data_partida)

        origem_iata = resultados[0]["iata"]

        if not data_partida:
            data_partida = get_default_departure_date()
        elif not validate_date(data_partida):
            return render_template("index.html", error="Formato de data inválido. Use AAAA-MM-DD.", preferences=prompt_usuario, origin=origem, date=data_partida)

        sugestoes, cidades = gerar_sugestoes_viagem(prompt_usuario)
        results = asyncio.run(fetch_hotels_and_flights(origem_iata, cidades, data_partida))

        # Buscar imagens para cada cidade
        imagens = {}
        for cidade in cidades:
            query = cidade["nome"]
            imagens[query] = buscar_imagens_pixabay(query)

        return render_template("results.html", suggestions=sugestoes, results=results, imagens=imagens)

    # Para requisições GET, buscar uma imagem aleatória
    imagem_fundo = buscar_imagem_aleatoria()
    return render_template("index.html", imagem_fundo=imagem_fundo)
if __name__ == "__main__":
    print(f" * Running on {public_url}")
    app.run()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

 * Running on https://4619-34-143-150-200.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 22:46:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 22:46:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 22:47:07] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 22:47:07] "GET /static/HI9M.gif HTTP/1.1" 200 -


buscar por locais com esportes de neve no continente asiatico Liste 3 de destinos turísticos, incluindo: - Informe cidades com serviço regular de voos que sejam proximas às atracoes destacadas: Nome da Cidade e seu codigo IATA em parentesis: (exemplo: 'San Juan (SJU)'). - Principais atrações e atividades. use no maximo 1000 palavras de forma que resuma bem o pedido do usuario.

1. **Niseko, Japão (CTS)**
   - Localizado na ilha de Hokkaido, Niseko é um destino popular para praticar esportes de neve. A cidade é conhecida por suas pistas de esqui e snowboard de vários níveis de dificuldade, além de oferecer atrações como a pista de esqui Niseko Village e a pista de esqui Grand Hirafu.
   - Além das atividades de esqui e snowboard, é possível explorar a cultura japonesa visitando os onsen (banhos termais), degustando o sushi local e conhecendo a vida tradicional dos ainu, o povo indígena da ilha.

2. **Gulmarg, Índia (IXG)**
   - Localizada no estado de Jammu e Caxemira, Gulmarg é um dos 

ERROR:__main__:Failed to fetch hotel details: 500
ERROR:__main__:Failed to fetch hotel details: 500
ERROR:__main__:Failed to fetch hotel details: 500
ERROR:__main__:Failed to fetch hotels: 400
ERROR:__main__:Failed to fetch hotel details: 500
ERROR:__main__:Failed to fetch hotel details: 500
ERROR:__main__:Failed to fetch hotel details: 500
ERROR:__main__:Failed to fetch hotel details: 500
ERROR:__main__:Failed to fetch hotel details: 500


Flights Data: [{'city': 'Japão', 'iata': 'CTS', 'hotels': [{'nome': 'BEST WESTERN FINO SAPPORO', 'id': 'BWCTS521', 'endereco': 'N/A', 'telefone': 'N/A', 'preco': 'N/A'}, {'nome': 'HILTON NISEKO VILLAGE', 'id': 'HLCTS101', 'endereco': 'N/A', 'telefone': 'N/A', 'preco': 'N/A'}, {'nome': 'THE HAMILTON SAPPORO', 'id': 'HSCTSAAC', 'endereco': 'N/A', 'telefone': 'N/A', 'preco': 'N/A'}], 'flights': [{'airline': 'CATHAY PACIFIC', 'flightNumber': '807', 'origin': 'ORD', 'destination': 'CTS', 'departureTime': '2025-03-19T15:40:00', 'arrivalTime': '2025-03-21T14:55:00', 'price': '568.00', 'currency': 'EUR'}, {'airline': 'EVA AIR', 'flightNumber': '55', 'origin': 'ORD', 'destination': 'CTS', 'departureTime': '2025-03-19T00:30:00', 'arrivalTime': '2025-03-20T14:05:00', 'price': '652.45', 'currency': 'EUR'}, {'airline': 'ASIANA AIRLINES', 'flightNumber': '6275', 'origin': 'ORD', 'destination': 'CTS', 'departureTime': '2025-03-19T07:00:00', 'arrivalTime': '2025-03-21T11:40:00', 'price': '972.70', 'cu

INFO:werkzeug:127.0.0.1 - - [09/Mar/2025 22:48:06] "POST /selecionar_cidade HTTP/1.1" 200 -


Imagens encontradas: {'Japão': ['https://pixabay.com/get/gd8a3ce3f8591191631027ab16bec0efe234c3d2cf3991fdb9e898381548d022b0dc5c4faebdf237a1f5060a6f153835a3461dadd45c2acfcc21346d554613887_1280.jpg', 'https://pixabay.com/get/g91368c896e2057000d4aa0022c1d2dfec7e01fdb715e342d7437eef4642f3c388fdaf5df1d39f843eb4b3ee58b8e0a854d0504e18b30da656e16c9bd6820c8fa_1280.jpg', 'https://pixabay.com/get/gf37a2a2379b609242aebf96317de1ea767ec3f756262bec3cf4620ea9cef97960557b034738c54ad6ea5a391fd9b2c8494f37d81f255fa92d3b5823c0f2e92c0_1280.jpg', 'https://pixabay.com/get/g6afa053f1499a7552f142617bf9a4efb37a324d07e91b31a2c1cba43bd70e14c16658ce5c355ab235d353ad6ad0497f1e69452f23466dfad3352878c395752aa_1280.jpg', 'https://pixabay.com/get/gd2c3438b49fd3c620155eb6854f4e43d3dec40b0f1830f471c1371be3fcc1f8ebb04f795180cb528fefe6391405cda13dcbad5b8994e65479f0df02f1f2a27fb_1280.jpg'], 'Índia': ['https://pixabay.com/get/gca03fb0bdd4273825ad50982e94b162720f581061425220d8f363a8c3e1ed19b791185f54ab40adf7a01b9ebe61b7577859a3